In [ ]:
import os

if not os.path.exists("lichess_db_puzzle.csv.zst"):
    !wget https://database.lichess.org/lichess_db_puzzle.csv.zst
if not os.path.exists("lichess_db_puzzle.csv"):
    !zstd -d lichess_db_puzzle.csv.zst

In [ ]:
import pandas as pd

# Read CSV and keep only important columns
df = pd.read_csv("lichess_db_puzzle.csv")
df = df[["FEN", "Rating", "Themes"]]

In [ ]:
# Create one hot encoding of the themes
one_hots = df["Themes"].str.get_dummies(sep=" ")
df = df.join(one_hots)

In [ ]:
# Now get bitboards for each piece type
import chess

pieces = [(t, col) for t in chess.PIECE_TYPES for col in chess.COLORS]
def fen_to_bitboards(fen):
    b = chess.BaseBoard(fen.split()[0])
    return pd.Series({chess.Piece(*p).symbol(): int(b.pieces(*p)) for p in pieces})

df = df.join(df["FEN"].apply(fen_to_bitboards))

In [ ]:
df = df.drop(columns=["FEN", "Themes"])

In [ ]:
df.to_csv("lichess_db_puzzle_processed.csv")